In [1]:
# TBD 1 : logger 추가
# TBD 2: flask github 참고, method, class, 파일의 맨 윗단 마다 pydoc 형식으로 달기
# TBD 3: 축약어를 자제할것 (특히 변수)

# -------------------------
#   done
# -------------------------

# 0. add data-setter, receiver system use python queue.Queue() class
# this will resolve i/o bottleneck
# 3. make iterable

# -------------------------
#   In Progress
# -------------------------

# 1. add logger
# 2. make image drawer overlay mask on image

# -------------------------
#   To be Done
# -------------------------

# 4. make verbose turn on and off
# 5. write pydoc

# python basic Module
import os
import sys
import types
import progressbar
from datetime import datetime
from shutil import copy
from pickle import dump, load

# math, image, plot Module
import numpy as np
import cv2
import matplotlib.pyplot as plt  # TBD

# tensorflow Module
import tensorflow as tf
from tensorflow.keras import backend as keras_backend
from tensorflow.keras.layers import GaussianNoise
from tensorflow.keras.layers import Input, Concatenate
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Nadam
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras import losses

# keras segmentaion third-party Moudle
import segmentation_models as sm

# custom Module
from gan_module.data_loader.medical_segmentation_data_loader import DataLoader
from gan_module.data_loader.manage_batch import BatchQueueManager

from gan_module.model.build_model import build_generator as build_generator
from gan_module.model.build_model import build_discriminator as build_discriminator
from gan_module import custom_loss
from gan_module.custom_loss import f1_loss_for_training, f1_score, dice_loss_for_training
from gan_module.util.manage_learning_rate import learning_rate_scheduler
from gan_module.util.draw_images import ImageDrawer
from gan_module.util.logger import TrainLogger
from gan_module.config import CONFIG

custom_loss.AXIS = [1, 2, 3]

USE_GPU = True

if USE_GPU:
    os.environ['CUDA_VISIBLE_DEVICES'] = '0'
    gpu_devices = tf.config.experimental.list_physical_devices("GPU")
    for device in gpu_devices:
        tf.config.experimental.set_memory_growth(device, True)
else:
    os.environ['CUDA_VISIBLE_DEVICES'] = '-1'


class Pix2PixSegmentation:
    def __init__(
        self,
        generator_power=32,
        discriminator_power=32,
        depth = None,
        generator_learning_rate=1e-4,
        discriminator_learning_rate=1e-4,
        temp_weights_path=".",
        on_memory=True,
        code_test=False
    ):
        # Input shape
        img_shape = CONFIG["img_shape"]
        input_channels = CONFIG["input_channels"]
        output_channels = CONFIG["output_channels"]

        self.input_img_shape = (*img_shape, input_channels)
        self.output_img_shape = (*img_shape, output_channels)
        # set parameter
        self.start_epoch = None
        self.on_memory = on_memory
        self.history = {"generator_loss": [],
                        "f1_loss_train": [], "f1_score_train": [],
                        "f1_loss_valid": [], "f1_score_valid": []}
        self.temp_weights_path = temp_weights_path

        # Configure data loader
        self.dataset_name = "tumor"
        self.data_loader = DataLoader(
            dataset_name=self.dataset_name,
            config_dict=CONFIG,
            on_memory=self.on_memory, 
            code_test=code_test
        )
        
        self.train_logger = TrainLogger()
        
        self.loaded_data_index = {
            "train": np.arange(self.data_loader.data_length["train"]),
            "valid": np.arange(self.data_loader.data_length["valid"])
        }
        
        # Configure Image Drawer
        self.image_drawer = ImageDrawer(
            dataset_name=self.dataset_name, data_loader=self.data_loader
        )
        self.discriminator_loss_ratio = keras_backend.variable(2.75)
        self.f1_loss_ratio = keras_backend.variable(97.25)
        self.discriminator_losses = np.array(
            [1 for _ in range(self.data_loader.data_length["train"])], dtype=np.float32)
        self.discriminator_acc_previous = 0.5
        self.discriminator_acces = np.array(
            [0.5 for _ in range(self.data_loader.data_length["train"])])
        self.discriminator_acces_previous = self.discriminator_acces.copy()
        self.generator_losses = np.array(
            [1 for _ in range(self.data_loader.data_length["train"])], dtype=np.float32)
        self.generator_losses_previous = self.generator_losses.copy()
        self.generator_f1_losses = np.array(
            [1 for _ in range(self.data_loader.data_length["train"])], dtype=np.float32)
        self.generator_loss_min = 100
        self.generator_loss_previous = 100
        self.generator_loss_max_previous = 1000
        self.total_f1_loss_min = 2
        self.weight_save_stack = False
        self.training_end_stack = 0
        # Calculate output shape of D (PatchGAN)
        self.disc_patch = (img_shape[0] // (2 ** depth), img_shape[1] // (2 ** depth), 1)
        # Number of filters in the first layer of G and D
        self.generator_learning_rate = generator_learning_rate
        self.discriminator_learning_rate = discriminator_learning_rate
        self.patience_count = 0
        generator_optimizer = Nadam(self.generator_learning_rate)
        discriminator_optimizer = Nadam(self.discriminator_learning_rate)
        
        # Build the generator
        self.generator = build_generator(
            input_img_shape=self.input_img_shape,
            output_channels=output_channels,
            generator_power=generator_power,
            depth=depth,
        )
        # Build and compile the discriminator
        self.discriminator = build_discriminator(
            input_img_shape=self.input_img_shape,
            output_img_shape=self.output_img_shape,
            discriminator_power=discriminator_power,
            depth=depth,
        )
        # self.discriminator = self.build_discriminator()
        # 'mse' or tf.keras.losses.Huber() tf.keras.losses.LogCosh()
        self.discriminator.compile(
            loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=0.1),
            optimizer=discriminator_optimizer,
            metrics=["accuracy"],
        )
        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # -------------------------
        # Construct Computational
        #   Graph of Generator
        # -------------------------

        # Input images and their conditioning images
        original_img = Input(shape=self.input_img_shape)
        masked_img = Input(shape=self.output_img_shape)
        # generate image from original_img for target masked_img
        model_masked_img = self.generator(original_img)
        
    
        # Discriminators determines validity of translated images / condition pairs
        model_validity = self.discriminator([original_img, model_masked_img])
        # give score by
        # 1. how generator trick discriminator
        # 2. how generator's image same as real photo in pixel
        # 3. if you want change loss, see doc https://keras.io/api/losses/
        # 4. 'mse', 'mae', tf.keras.losses.LogCosh(),  tf.keras.losses.Huber()
        self.combined = Model(
            inputs=[original_img, masked_img],
            outputs=[model_validity, model_masked_img],
        )
        
        self.combined.compile(
            loss=[
                tf.keras.losses.BinaryCrossentropy(label_smoothing=0.1),
                dice_loss_for_training
            ],
            loss_weights=[2.75, 97.25],
            optimizer=generator_optimizer,
        )

    def train(self, epochs, batch_size=1, epoch_shuffle_term=10):

        start_time = datetime.now()

        # Adversarial loss ground truths
        self.training_end_stack = 0
        self.batch_size = batch_size
        valid_patch = np.ones(
            (self.batch_size, *self.disc_patch), dtype=np.float32)
        fake_patch = np.zeros(
            (self.batch_size, *self.disc_patch), dtype=np.float32)
        # TBD : move batch_queue_manager to __init__
        self.batch_queue_manager = BatchQueueManager(self, batch_size, self.on_memory)

        if self.start_epoch is None:
            self.start_epoch = 0
        for epoch in range(self.start_epoch, epochs):
            batch_i = 0

            generator_loss_max_in_epoch = 0
            generator_loss_min_in_epoch = 1000
            generator_discriminator_losses = np.array(
            [1 for _ in range(self.data_loader.data_length["train"])], dtype=np.float32)
            # shffle data maybe
            if epoch % epoch_shuffle_term == 0:
                self.data_loader.shuffle_train_imgs()

            if self.discriminator_acc_previous < 0.75:
                discriminator_learning = True
            else:
                discriminator_learning = False
                
            generator_current_learning_rate = learning_rate_scheduler(
                self.generator_learning_rate,
                epoch+self.patience_count,
            )
            discriminator_current_learning_rate = learning_rate_scheduler(
                self.discriminator_learning_rate,
                epoch+self.patience_count,
            ) * (1 - self.discriminator_acc_previous)
            keras_backend.set_value(
                self.discriminator.optimizer.learning_rate,
                discriminator_current_learning_rate,
            )
            keras_backend.set_value(
                self.discriminator.optimizer.learning_rate,
                discriminator_current_learning_rate,
            )
            keras_backend.set_value(
                self.discriminator_loss_ratio,
                keras_backend.variable(0.5) + 4.5 * self.discriminator_acc_previous,
            )
            keras_backend.set_value(
                self.f1_loss_ratio,
                keras_backend.variable(99.5) - 4.5 * self.discriminator_acc_previous
            )
            bar = progressbar.ProgressBar(
                maxval=self.data_loader.data_length["train"]).start()
            while batch_i + self.batch_size <= self.data_loader.data_length["train"]:

                batch_index = self.loaded_data_index["train"][batch_i: batch_i +
                                                              self.batch_size]

                original_img, masked_img = self.batch_queue_manager.get_batch(
                    data_mode="train")
                model_masked_img = self.generator.predict_on_batch(
                    original_img)

                self.original_img = original_img
                self.masked_img = masked_img
                # ---------------------
                #  Train Discriminator
                # ---------------------
                # Train Discriminator for valid image if it failed to detect fake image

                if discriminator_learning:
                    self.discriminator.train_on_batch(
                        [original_img, masked_img], valid_patch)

                batch_discriminator_acc_previous = np.mean(
                    self.discriminator_acces_previous[batch_index])

                # -----------------
                #  Train Generator
                # -----------------

                # Train the generators

                generator_loss = self.combined.train_on_batch(
                    [original_img, masked_img],
                    [valid_patch, masked_img]
                )
                # train discriminator for fake image if it failed to detect fake image
                if (batch_discriminator_acc_previous <= 0.5 or epoch == 0) and discriminator_learning:
                    discriminator_loss = self.discriminator.train_on_batch(
                        [original_img, model_masked_img], fake_patch)
                else:
                    discriminator_loss = self.discriminator.test_on_batch(
                        [original_img, model_masked_img], fake_patch)

                self.discriminator_losses[batch_index] = discriminator_loss[0]
                self.discriminator_acces[batch_index] = discriminator_loss[1]
                self.generator_losses[batch_index] = generator_loss[0]
                self.generator_f1_losses[batch_index] = generator_loss[2]
                generator_discriminator_losses = generator_loss[1]
                # plot progress
                bar.update(batch_i)
                
                # 한 배치 끝
                batch_i += self.batch_size
            
            
            # training batch 사이클 끝
            
            self.train_logger.write_log(
                f"{epoch}/{epochs} ({epoch+self.patience_count})",
                np.mean(self.discriminator_acces),
                np.mean(self.generator_losses),
                np.max(self.generator_losses),
                np.min(self.generator_losses),
                f"{self.generator_loss_min - np.mean(self.generator_losses)}({np.mean(self.generator_losses) / self.generator_loss_min})",
                self.generator_loss_min,
                generator_current_learning_rate,
                datetime.now() - start_time
            )
            self.image_drawer.sample_images(
                self.generator, epoch)
            
            if np.mean(self.generator_losses) / self.generator_loss_min < 1.05:
                #if self.generator_loss_min > np.mean(self.generator_losses):
                valid_f1_loss_list = []
                for index in range(0, self.data_loader.data_length["valid"], self.batch_size):

                    valid_source_img, valid_masked_img = self.batch_queue_manager.get_batch(
                        data_mode="valid")

                    valid_model_masked_img = self.generator.predict_on_batch(
                        valid_source_img)

                    valid_f1_loss = f1_loss_for_training(
                        valid_masked_img, valid_model_masked_img)
                    valid_f1_loss_list.append(valid_f1_loss)
                # compute valid_f1_loss end    
                total_f1_loss = np.mean(self.generator_f1_losses) + np.mean(valid_f1_loss_list)

                print(f"discriminator_loss : {np.mean(self.discriminator_losses)}")
                print(f"generator_discriminator_loss : {np.mean(generator_discriminator_losses)}")
                print(f"train_f1_loss : {np.mean(self.generator_f1_losses)}")
                print(
                    f"valid_f1_loss : {np.mean(valid_f1_loss_list)}")
                print(f"current/min total_f1_loss = {total_f1_loss} / {self.total_f1_loss_min}")
                if self.generator_loss_min > np.mean(self.generator_losses):
                    self.generator_loss_min = np.mean(self.generator_losses)
                    self.generator_loss_max_min = generator_loss_max_in_epoch
                    self.generator_loss_min_min = generator_loss_min_in_epoch
                
                if self.total_f1_loss_min > total_f1_loss: 
                    self.total_f1_loss_min = total_f1_loss
                    self.save_study_info()
                    self.weight_save_stack = True
                    print("save weights")

            else:
                print("loss decrease.")
                self.patience_count += 5
                self.load_best_weights()

            # previous generator_loss 갱신
            self.generator_loss_previous = np.mean(self.generator_losses)
            self.generator_loss_max_previous = np.max(self.generator_losses)

            if epoch >= 10 and self.weight_save_stack:
                copy(
                    "generator.h5",
                    "./generator_weights/generator_"
                    + str(round(self.generator_loss_min, 5))
                    + "_"
                    + str(round(self.generator_loss_max_min, 5))
                    + ".h5",
                )
                self.weight_save_stack = False

            self.discriminator_acc_previous = np.mean(self.discriminator_acces)
            self.discriminator_acces_previous = self.discriminator_acces.copy()
            self.generator_losses_previous = self.generator_losses.copy()
            # TBD: add epoch bigger than history length
            self.history["generator_loss"].append(
                np.mean(self.generator_losses))
            self.history["f1_loss_train"].append(
                np.mean(self.generator_f1_losses))
            self.history["f1_loss_valid"].append(
                np.mean(valid_f1_loss_list))

    def get_info_folderPath(self):
        return (
            str(round(self.generator_loss_min, 5))
            + "_"
            + str(round(self.generator_loss_max_min, 5))
        )

    def save_study_info(self, path=None):

        if path is None:
            path = self.temp_weights_path

        generator_weigth_path = os.path.join(path, "generator.h5")
        discriminator_weigth_path = os.path.join(path, "discriminator.h5")
        combined_weigth_path = os.path.join(path, "combined.h5")

        self.generator.save_weights(generator_weigth_path)
        self.discriminator.save_weights(discriminator_weigth_path)
        self.combined.save_weights(combined_weigth_path)

        study_info = {}
        study_info["start_epoch"] = self.start_epoch
        study_info["train_loaded_data_index"] = self.loaded_data_index["train"]
        study_info["generator_loss_min"] = self.generator_loss_min
        study_info["generator_loss_max_min"] = self.generator_loss_max_min
        study_info["generator_loss_min_min"] = self.generator_loss_min_min
        study_info["generator_losses_previous"] = self.generator_losses_previous
        study_info["discriminator_acces"] = self.discriminator_acces
        study_info["history"] = self.history
        file = open(path + "/study_info.pkl", "wb")
        dump(study_info, file)
        file.close()

    def load_study_info(self):

        self.generator.load_weights("generator.h5")
        self.discriminator.load_weights("discriminator.h5")
        self.combined.load_weights("combined.h5")

        if os.path.isfile("study_info.pkl"):
            file = open("study_info.pkl", "rb")
            study_info = load(file)
            file.close()
            self.start_epoch = study_info["start_epoch"]
            self.loaded_data_index["train"] = study_info["train_loaded_data_index"]
            self.generator_loss_min = study_info["generator_loss_min"]
            self.generator_loss_max_min = study_info["generator_loss_max_min"]
            self.generator_loss_min_min = study_info["generator_loss_min_min"]
            self.generator_losses_previous = study_info["generator_losses_previous"]
            self.discriminator_acces = study_info["discriminator_acces"]
            self.history = study_info["history"]
        else:
            print("No info pkl file!")

    def load_best_weights(self):
        self.generator.load_weights(self.temp_weights_path + "/generator.h5")
        self.discriminator.load_weights(
            self.temp_weights_path + "/discriminator.h5")
        self.combined.load_weights(self.temp_weights_path + "/combined.h5")


Segmentation Models: using `tf.keras` framework.
{'img_shape': [512, 512], 'input_channels': 3, 'output_channels': 1}


In [2]:
generator_lr = 1e-3
discriminator_lr = 1e-3
batch_size = 4
g_lr = generator_lr * batch_size
d_lr = discriminator_lr * batch_size
gan = Pix2PixSegmentation(generator_power=4, discriminator_power=4, depth = 5,
                          generator_learning_rate=g_lr, discriminator_learning_rate=d_lr,
                          on_memory=True, code_test=False)

In [3]:
#gan.load_study_info()
# gan.start_epoch = 30
gan.train(epochs=325, batch_size=batch_size, epoch_shuffle_term=100)

 99% (6196 of 6200) |################### | Elapsed Time: 0:21:12 ETA:   0:00:02{
2021-04-08 08:14:30,605 - train - INFO - 
Epoch : 0/325 (0)
Discriminator_acces : 0.28074092741935486
Mean generator loss : 73.04253387451172
Max generator loss : 113.3912124633789
Min generator loss : 20.771621704101562
Generator loss decrease : 26.95746612548828(0.7304253387451172)
Current lowest generator loss : 100
Current Learning_rate : 0.0002
Elapsed_time : 0:21:18.409690
}


discriminator_loss : 3.6891138553619385
generator_discriminator_loss : 0.6870644092559814
train_f1_loss : 0.6936622858047485
valid_f1_loss : 0.6117153168318407
current/min total_f1_loss = 1.305377602636589 / 2


N/A% (0 of 6200) |                       | Elapsed Time: 0:00:00 ETA:  --:--:--

save weights


 99% (6196 of 6200) |################### | Elapsed Time: 0:20:04 ETA:   0:00:00{
2021-04-08 08:34:55,329 - train - INFO - 
Epoch : 1/325 (1)
Discriminator_acces : 0.2459280493951613
Mean generator loss : 53.383514404296875
Max generator loss : 95.57025909423828
Min generator loss : 20.959228515625
Generator loss decrease : 19.659019470214844(0.7308551669120789)
Current lowest generator loss : 73.04253387451172
Current Learning_rate : 0.0004
Elapsed_time : 0:41:43.135097
}


...

discriminator_loss : 0.8432908058166504
generator_discriminator_loss : 0.3284015655517578
train_f1_loss : 0.381191611289978
valid_f1_loss : 0.3858494834156745
current/min total_f1_loss = 0.7670410947056525 / 0.7570281481043557


 99% (6196 of 6200) |################### | Elapsed Time: 0:18:18 ETA:   0:00:00{
2021-04-08 12:50:13,348 - train - INFO - 
Epoch : 14/325 (14)
Discriminator_acces : 0.0
Mean generator loss : 39.46062469482422
Max generator loss : 88.99407958984375
Min generator loss : 16.486003875732422
Generator loss decrease : 0.5990715026855469(0.9850455522537231)
Current lowest generator loss : 40.059696197509766
Current Learning_rate : 0.003
Elapsed_time : 4:57:01.153695
}
N/A% (0 of 6200) |                       | Elapsed Time: 0:00:00 ETA:  --:--:--

discriminator_loss : 1.5267332792282104
generator_discriminator_loss : 0.42084547877311707
train_f1_loss : 0.3808203935623169
valid_f1_loss : 0.38247340933914104
current/min total_f1_loss = 0.763293802901458 / 0.7570281481043557


  1% (76 of 6200) |                      | Elapsed Time: 0:00:15 ETA:   0:19:48

KeyboardInterrupt: 

In [20]:
gan.original_img

<tf.Tensor: shape=(4, 512, 512, 3), dtype=float64, numpy=
array([[[[ 0.89019608,  0.60784316,  0.82745099],
         [ 0.90588236,  0.66274512,  0.83529413],
         [ 0.92156863,  0.7019608 ,  0.84313726],
         ...,
         [ 0.93725491,  0.78039217,  0.88235295],
         [ 0.92941177,  0.77254903,  0.86666667],
         [ 0.9137255 ,  0.77254903,  0.88235295]],

        [[ 0.90588236,  0.62352943,  0.84313726],
         [ 0.9137255 ,  0.66274512,  0.84313726],
         [ 0.92156863,  0.69411767,  0.8509804 ],
         ...,
         [ 0.99215686,  0.88235295,  0.97647059],
         [ 0.96862745,  0.81960785,  0.92941177],
         [ 0.87450981,  0.73333335,  0.84313726]],

        [[ 0.92941177,  0.63137257,  0.84313726],
         [ 0.92156863,  0.65490198,  0.8509804 ],
         [ 0.90588236,  0.67058825,  0.85882354],
         ...,
         [ 1.        ,  0.86666667,  0.99215686],
         [ 0.97647059,  0.81176472,  0.92941177],
         [ 0.88235295,  0.72549021,  0.8509804

In [19]:
temp = gan.data_loader.loaded_data_object["train"].values()

for index, (input_img, output_img) in enumerate(zip(*temp)):
    print(index)
    if index > 40:
        break
    print(index)
    print(input_img.shape)
    print(output_img.shape)
    

0
0
(512, 512, 3)
(512, 512, 1)
1
1
(512, 512, 3)
(512, 512, 1)
2
2
(512, 512, 3)
(512, 512, 1)
3
3
(512, 512, 3)
(512, 512, 1)
4
4
(512, 512, 3)
(512, 512, 1)
5
5
(512, 512, 3)
(512, 512, 1)
6
6
(512, 512, 3)
(512, 512, 1)
7
7
(512, 512, 3)
(512, 512, 1)
8
8
(512, 512, 3)
(512, 512, 1)
9
9
(512, 512, 3)
(512, 512, 1)
10
10
(512, 512, 3)
(512, 512, 1)
11
11
(512, 512, 3)
(512, 512, 1)
12
12
(512, 512, 3)
(512, 512, 1)
13
13
(512, 512, 3)
(512, 512, 1)
14
14
(512, 512, 3)
(512, 512, 1)
15
15
(512, 512, 3)
(512, 512, 1)
16
16
(512, 512, 3)
(512, 512, 1)
17
17
(512, 512, 3)
(512, 512, 1)
18
18
(512, 512, 3)
(512, 512, 1)
19
19
(512, 512, 3)
(512, 512, 1)


In [5]:
import time

temp_source = gan.original_img
temp_mask = gan.masked_img

start_time = time.time()
gan.generator.train_on_batch(temp_source, temp_mask)
print(f"elapsed time : {time.time() - start_time}")

temp_source = tf.convert_to_tensor(temp_source)
temp_mask = tf.convert_to_tensor(temp_mask)

start_time = time.time()
gan.generator.train_on_batch(temp_source, temp_mask)
print(f"elapsed time : {time.time() - start_time}")

elapsed time : 4.586500883102417
elapsed time : 0.29304075241088867


# Iterator : 260초
# Queue Iterator : 200초

In [5]:
import time
import threading
from queue import Queue

ITER_NUM = 620
batch_size = 10

gan.generator.compile(
    loss=sm.losses.BinaryFocalLoss(),
    optimizer=Nadam(gan.generator_learning_rate),
    metrics=["accuracy"],
)

def batch_setter(queue):
    batch_i = 0
    count = 0
    while batch_i + gan.batch_size <= gan.data_loader.train_data_length and count < ITER_NUM:
        
        batch_index = gan.train_loaded_data_index[batch_i: batch_i +
                                                   gan.batch_size]        
        
        batch_tuple = gan.data_loader.get_data(
        data_mode="train", index=batch_index)

        queue.put(batch_tuple)
        queue.join()
        count += 1
    
def batch_getter(queue):
    
    original_img, masked_img = queue.get()
    tensor_original_img = tf.convert_to_tensor(original_img)
    tensor_masked_img = tf.convert_to_tensor(masked_img)
    queue.task_done()
    
    return tensor_original_img, tensor_masked_img
    
def batch_trainer(original_img, masked_img):
    
    gan.generator.train_on_batch(temp_source, temp_mask)

q = Queue()

setter = threading.Thread(target=batch_setter, args=(q,),daemon=True)
setter.start()
start_time = time.time()
for i in range(ITER_NUM):
    tensor_original_img, tensor_masked_img = batch_getter(q)
    
    gan.generator.train_on_batch(tensor_original_img, tensor_masked_img)
print(f"elapsed time : {time.time() - start_time}")

elapsed time : 204.91554951667786


In [29]:
start_time = time.time()
batch_i = 0
count = 0
while batch_i + gan.batch_size <= gan.data_loader.train_data_length and count < ITER_NUM:

    batch_index = gan.train_loaded_data_index[batch_i: batch_i +
                                               gan.batch_size]        
    batch_tuple = gan.data_loader.get_data(
    data_mode="train", index=batch_index)
    
    gan.generator.train_on_batch(*batch_tuple)
    
    count += 1
print(f"elapsed time : {time.time() - start_time}")

elapsed time : 260.5594081878662


In [10]:
import cv2

temp = tensor_masked_img
print(type(temp))
print(type(temp.numpy()))

<class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'numpy.ndarray'>


In [11]:
isinstance(temp.numpy(), tf.Tensor)

False

In [1]:
from gan_module.model.build_model import build_dual_discriminator

temp = build_dual_discriminator(
            input_img_shape=(512,512,3),
            output_img_shape=(512,512,1),
            discriminator_power=1,
)

In [2]:
import tensorflow as tf
from tensorflow.keras.optimizers import Nadam

temp.compile(
    loss=[
        tf.keras.losses.BinaryCrossentropy(label_smoothing=0.1),
        tf.keras.losses.BinaryCrossentropy(label_smoothing=0.1)
    ],
    optimizer=Nadam(),
    metrics=["accuracy"],
)

In [3]:
import numpy as np

image_mockup = np.ones((1,512,512,3))
mask_mockup = np.ones((1,512,512,1))
patch_mockup = np.ones((1,8,8,1))

temp.test_on_batch([image_mockup, mask_mockup], [patch_mockup, patch_mockup])

UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node functional_1/conv2d_34/Conv2D (defined at <ipython-input-3-d8b1761e412e>:7) ]] [Op:__inference_test_function_13463]

Function call stack:
test_function
